In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
torch.manual_seed(1)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [3]:
torch.cuda.empty_cache() 

# Create Train and Validation

In [4]:
X = []
Y = []
for p in tqdm(np.arange(0, 50), total = 50):
    X_p = torch.load('data_processed/1face_X_part' + str(p) + '.pt', map_location = device)
    Y_p = torch.load('data_processed/1face_Y_part' + str(p) + '.pt', map_location = device)
    X = X + X_p
    Y = Y + Y_p

100%|██████████| 50/50 [01:17<00:00,  1.71s/it]


In [5]:
len(X)

104343

In [6]:
len(Y)

104343

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X[0:104300], Y[0:104300], 
                                                  test_size=4300, random_state=42, stratify = Y[0:104300])
X_train = torch.stack(X_train)
X_val = torch.stack(X_val)

In [8]:
X_train.shape

torch.Size([100000, 30, 512])

In [9]:
from torch.utils.data import TensorDataset, DataLoader
train_data = TensorDataset(X_train, torch.from_numpy(np.array(Y_train)))
val_data = TensorDataset(X_val, torch.from_numpy(np.array(Y_val)))


In [20]:
train_batch_size = 1000
val_batch_size = 100

train_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights = Y_train, num_samples = train_batch_size)

train_loader = DataLoader(train_data, shuffle=False, batch_size=train_batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=val_batch_size)

# Create Model

In [35]:
import torch.nn as nn

class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)

        self.dropout = nn.Dropout(0.5)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_dim, 8)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(8, output_size)
        torch.nn.init.xavier_uniform_(self.fc4.weight)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.float()
        
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.batchnorm(out)
        out = self.fc1(out)
        out = self.elu(out)
#         out = self.fc2(out)
#         out = self.elu(out)
#         out = self.fc3(out)
#         out = self.elu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [36]:
input_size = 512
output_size = 1
hidden_dim = 64
n_layers = 5

model = DFDCNet(input_size, output_size, hidden_dim, n_layers)
model.to(device)
train_criterion = nn.BCELoss()
val_criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)

DFDCNet(
  (lstm): LSTM(512, 64, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu): ELU(alpha=1.0)
  (fc1): Linear(in_features=64, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [37]:
# scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=250, verbose=True)


In [38]:
epochs = 10000
counter = 0
print_every = 1000
clip = .5
valid_loss_min = np.Inf
val_loss = torch.tensor(np.Inf)
model.train()
for i in range(epochs):
    h = model.init_hidden(train_batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = train_criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(val_batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = val_criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './model_1face_sample.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
    scheduler.step(val_loss.item())


Epoch: 10/10000... Step: 1000... Loss: 0.470699... Val Loss: 0.449794
Validation loss decreased (inf --> 0.449794).  Saving model ...
Epoch: 20/10000... Step: 2000... Loss: 0.470133... Val Loss: 0.449347
Validation loss decreased (0.449794 --> 0.449347).  Saving model ...
Epoch: 30/10000... Step: 3000... Loss: 0.469323... Val Loss: 0.449279
Validation loss decreased (0.449347 --> 0.449279).  Saving model ...
Epoch: 40/10000... Step: 4000... Loss: 0.470698... Val Loss: 0.449459
Epoch: 50/10000... Step: 5000... Loss: 0.469721... Val Loss: 0.449458


KeyboardInterrupt: 

# Reference
* https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb